# 작업형2
- 빅데이터 분석기사 2회 실기
- 데이터 다운로드 : https://www.kaggle.com/prachi13/customer-analytics
- 데이터 파일 명 : Train.csv (파일명이 다를 경우 파일명을 수정해주세요)
- (데이터가 삭제될 시 수업노트에 유사 데이터 셋을 올려두겠습니다)


## 제품 배송 시간에 맞춰 배송되었는지 예측모델 만들기
- 제공된 데이터
    - X_train.csv
    - y_train.csv
    - X_test.csv

- 학습용 데이터 (X_train, y_train)을 이용하여 배송 예측 모형을 만든 후, 이를 평가용 데이터(X_test)에 적용하여 얻은 예측(시간에 맞춰 도착하지 않을 확률)값을 다음과 같은 형식의 CSV파일로 생성하시오(제출한 모델의 성능은 ROC-AUC 평가지표에 따라 채점)
- 0 정시 도착, 1 정시 도착하지 않음

```
ID,Reached.on.Time_Y.N
3500,0.267
3501,0.578
3502,0.123

```

- 출력을 원하실 경우 print() 함수 활용
- 예시) print(df.head())
- getcwd(), chdir() 등 작업 폴더 설정 불필요
- 파일 경로 상 내부 드라이브 경로(C: 등) 접근 불가

- 데이터 파일 읽기 예제

```
# 데이터 불러오기
import pandas as pd
X_test = pd.read_csv("data/X_test.csv")
X_train = pd.read_csv("data/X_train.csv")
y_train = pd.read_csv("data/y_train.csv")
```

- csv파일 생성 예시

```
pd.DataFrame({'ID': X_test['cust_id'], 'Reached.on.Time_Y.N': pred}).to_csv('003000000.csv', index=False)
```


In [ ]:
# 시험환경 세팅 3개 파일 생성 (코드 변경 X)
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

def exam_data_load(df, target, id_name="", null_name=""):
    if id_name == "":
        df = df.reset_index().rename(columns={"index": "id"})
        id_name = 'id'
    else:
        id_name = id_name

    if null_name != "":
        df[df == null_name] = np.nan

    X_train, X_test = train_test_split(df, test_size=0.2, random_state=2022)
    y_train = X_train[[id_name, target]]
    X_train = X_train.drop(columns=[target])

    y_test = X_test[[id_name, target]]
    X_test = X_test.drop(columns=[target])
    return X_train, X_test, y_train, y_test

df = pd.read_csv("Train.csv") # 파일명이 다를 경우 파일명을 수정해주세요
X_train, X_test, y_train, y_test = exam_data_load(df, target='Reached.on.Time_Y.N', id_name='ID')
X_train.to_csv("X_train.csv", index=False)
y_train.to_csv("y_train.csv", index=False)
X_test.to_csv("X_test.csv", index=False)

In [ ]:
# 데이터 불러오기
import pandas as pd
X_test = pd.read_csv("X_test.csv")
X_train = pd.read_csv("X_train.csv")
y_train = pd.read_csv("y_train.csv")

print(X_train.shape, y_train.shape, X_test.shape)
# (8799, 11) (8799, 2) (2200, 11), y_train 컬럼은 2개



(8799, 11) (8799, 2) (2200, 11)


## EDA

In [ ]:
# 데이터 확인(X_train)
print(X_train.head()) # 수치 + 범주형 혼합되어 있음을 알 수 있음.


      ID Warehouse_block Mode_of_Shipment  Customer_care_calls  \
0   8440               B             Ship                    4   
1  10849               D             Ship                    3   
2   9478               B             Ship                    4   
3   2968               B             Ship                    6   
4   6964               B             Road                    5   

   Customer_rating  Cost_of_the_Product  Prior_purchases Product_importance  \
0                1                  284                4                low   
1                2                  201                4             medium   
2                4                  181                3             medium   
3                2                  236                4                low   
4                1                  282                4                low   

  Gender  Discount_offered  Weight_in_gms  
0      F                 4           4795  
1      M                10           556

In [ ]:
# 데이터 확인(y_train)
print(y_train.head())
# 1은 정시 도착 X, 0은 정시 도착하였음.

      ID  Reached.on.Time_Y.N
0   8440                    1
1  10849                    0
2   9478                    1
3   2968                    1
4   6964                    0


In [ ]:
# 결측치 확인 train
print(X_train.isnull().sum())
print(y_train.isnull().sum())

# X와 Y에 모두 결측치 없음.

ID                     0
Warehouse_block        0
Mode_of_Shipment       0
Customer_care_calls    0
Customer_rating        0
Cost_of_the_Product    0
Prior_purchases        0
Product_importance     0
Gender                 0
Discount_offered       0
Weight_in_gms          0
dtype: int64
ID                     0
Reached.on.Time_Y.N    0
dtype: int64


In [ ]:
# 결측치 확인 test
print(X_test.isnull().sum())
# X-test 결측치 없음.

ID                     0
Warehouse_block        0
Mode_of_Shipment       0
Customer_care_calls    0
Customer_rating        0
Cost_of_the_Product    0
Prior_purchases        0
Product_importance     0
Gender                 0
Discount_offered       0
Weight_in_gms          0
dtype: int64


In [ ]:
# 타겟(레이블) 확인 0 정시 도착, 1 정시 도착하지 않음
print(y_train['Reached.on.Time_Y.N'].value_counts())

Reached.on.Time_Y.N
1    5236
0    3563
Name: count, dtype: int64


In [ ]:
# 데이터 타입 확인
print(X_train.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8799 entries, 0 to 8798
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   ID                   8799 non-null   int64 
 1   Warehouse_block      8799 non-null   object
 2   Mode_of_Shipment     8799 non-null   object
 3   Customer_care_calls  8799 non-null   int64 
 4   Customer_rating      8799 non-null   int64 
 5   Cost_of_the_Product  8799 non-null   int64 
 6   Prior_purchases      8799 non-null   int64 
 7   Product_importance   8799 non-null   object
 8   Gender               8799 non-null   object
 9   Discount_offered     8799 non-null   int64 
 10  Weight_in_gms        8799 non-null   int64 
dtypes: int64(7), object(4)
memory usage: 756.3+ KB
None


In [ ]:
# 기초 통계 획인
print(X_train.describe())

                 ID  Customer_care_calls  Customer_rating  \
count   8799.000000          8799.000000      8799.000000   
mean    5514.696556             4.051369         2.991817   
std     3175.107399             1.141739         1.411938   
min        1.000000             2.000000         1.000000   
25%     2749.500000             3.000000         2.000000   
50%     5519.000000             4.000000         3.000000   
75%     8253.000000             5.000000         4.000000   
max    10999.000000             7.000000         5.000000   

       Cost_of_the_Product  Prior_purchases  Discount_offered  Weight_in_gms  
count          8799.000000      8799.000000       8799.000000    8799.000000  
mean            209.800318         3.561882         13.293215    3633.145130  
std              48.186779         1.518472         16.112977    1637.717073  
min              96.000000         2.000000          1.000000    1001.000000  
25%             169.000000         3.000000          4.

In [ ]:
# 기초 통계 획인 object
print(X_train.describe(include = 'object'))
cols = X_train.select_dtypes(include='object').columns
print(X_train[cols].nunique())

       Warehouse_block Mode_of_Shipment Product_importance Gender
count             8799             8799               8799   8799
unique               5                3                  3      2
top                  F             Ship                low      F
freq              2929             5972               4225   4450
Warehouse_block       5
Mode_of_Shipment      3
Product_importance    3
Gender                2
dtype: int64


In [ ]:
# 기초 통계 획인 object
print(X_test.describe(include = 'object'))

       Warehouse_block Mode_of_Shipment Product_importance Gender
count             2200             2200               2200   2200
unique               5                3                  3      2
top                  F             Ship                low      M
freq               737             1490               1072   1105


In [ ]:
# [Tip] object 타입 고유값 개수 확인
print(X_test[cols].nunique())

Warehouse_block       5
Mode_of_Shipment      3
Product_importance    3
Gender                2
dtype: int64


In [ ]:
# X데이터와 y데이터 합치기
df = pd.concat([X_train, y_train['Reached.on.Time_Y.N']], axis = 1)
print(df)

# df에 가장 마지막 target값이 합쳐짐.

         ID Warehouse_block Mode_of_Shipment  Customer_care_calls  \
0      8440               B             Ship                    4   
1     10849               D             Ship                    3   
2      9478               B             Ship                    4   
3      2968               B             Ship                    6   
4      6964               B             Road                    5   
...     ...             ...              ...                  ...   
8794   6385               D             Ship                    3   
8795   4721               C             Ship                    4   
8796    174               F             Ship                    3   
8797   1245               A           Flight                    4   
8798   4990               B             Ship                    2   

      Customer_rating  Cost_of_the_Product  Prior_purchases  \
0                   1                  284                4   
1                   2                  201   

In [ ]:
# idea? 할인율이 도착에 영향을 미칠까?
cond = df['Discount_offered'] >10
sum(cond) #2104

2104

In [ ]:
sum(df[cond]['Reached.on.Time_Y.N']) #2104
# 따라서 할인율이 10% 이상인 경우는 모두 제시간에 도착하지 않았음을 알 수 있음.
# 할인율이 높을 수록 배송이 지연될 확률이 오를 수 있다.

2104


## 데이터 전처리 및 피처엔지니어링

In [ ]:
# 라벨 인코딩
from sklearn.preprocessing import LabelEncoder
cols = X_train.select_dtypes(include = 'object').columns

for col in cols:
  le = LabelEncoder()
  X_train[col] = le.fit_transform(X_train[col])
  X_test[col] = le.transform(X_test[col])

print(X_train.head(5))

      ID  Warehouse_block  Mode_of_Shipment  Customer_care_calls  \
0   8440                1                 2                    4   
1  10849                3                 2                    3   
2   9478                1                 2                    4   
3   2968                1                 2                    6   
4   6964                1                 1                    5   

   Customer_rating  Cost_of_the_Product  Prior_purchases  Product_importance  \
0                1                  284                4                   1   
1                2                  201                4                   2   
2                4                  181                3                   2   
3                2                  236                4                   1   
4                1                  282                4                   1   

   Gender  Discount_offered  Weight_in_gms  
0       0                 4           4795  
1       1           

In [ ]:
# trainID 삭제, testID 값만 옮겨둠
X_train_id = X_train.pop('ID')
X_test_id = X_test.pop('ID')
X_test.head()

,Warehouse_block,Mode_of_Shipment,Customer_care_calls,Customer_rating,Cost_of_the_Product,Prior_purchases,Product_importance,Gender,Discount_offered,Weight_in_gms
0,2,2,6,5,229,8,2,0,2,4271
1,1,2,7,3,306,4,1,0,4,1630
2,2,2,4,5,148,3,2,0,5,5216
3,2,0,4,4,236,3,1,1,10,4347
4,1,2,3,4,194,3,2,1,3,2169


## 모델 및 평가

In [ ]:
# 라이브러리 불러오기
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score

In [ ]:
# 검증 데이터 분리
from sklearn.model_selection import train_test_split
X_tr, X_val, y_tr, y_val = train_test_split(
    X_train, y_train['Reached.on.Time_Y.N'], test_size = 0.2, random_state = 2022)
    # 순서는 X 검증 데이터 / Y의 타겟 데이터 (이 문제의 경우는 제대로 도착했는지의 여부 )
print(X_tr.shape, X_val.shape, y_tr.shape, y_val.shape)

(7039, 10) (1760, 10) (7039,) (1760,)


In [ ]:
# 로지스틱 회귀
model = LogisticRegression(random_state = 2022)
model.fit(X_tr, y_tr)
pred = model.predict_proba(X_val)
print(roc_auc_score(y_val, pred[:,1]))

0.7297546090137466


In [ ]:
# 의사결정나무
model = DecisionTreeClassifier(random_state = 2022)
model.fit(X_tr, y_tr)
pred = model.predict_proba(X_val)
print(roc_auc_score(y_val, pred[:,1]))

0.6384948675914754


In [ ]:
# 랜덤포레스트
model = RandomForestClassifier()
model.fit(X_tr, y_tr)
pred = model.predict_proba(X_val)
print(roc_auc_score(y_val, pred[:,1])) # 0.7598710184253666

0.7548254526847111


In [ ]:
# XGBOOST
model = XGBClassifier()
model.fit(X_tr, y_tr)
pred = model.predict_proba(X_val)
print(roc_auc_score(y_val, pred[:,1]))

0.7496656552822458


## 예측 및 제출

In [ ]:
# 예측
pred = model.predict_proba(X_test)
print(pred)

[[0.67 0.33]
 [0.61 0.39]
 [0.68 0.32]
 ...
 [0.53 0.47]
 [0.66 0.34]
 [0.4  0.6 ]]


In [ ]:
# 데이터프레임 만들기
submit = pd.DataFrame({
    'ID' : X_test_id,
    "Reached.on.Time_Y.N": pred[:,1]
})
print(submit)

         ID  Reached.on.Time_Y.N
0      8285                 0.33
1     10192                 0.39
2      8675                 0.32
3      5753                 0.38
4       448                 0.93
...     ...                  ...
2195   7193                 0.48
2196   4445                 0.29
2197   7390                 0.47
2198   9977                 0.34
2199   5696                 0.60

[2200 rows x 2 columns]


In [ ]:
# CSV파일 만들기
submit.to_csv('20240611.csv', index=False)

##  제대로 제출 되었는지 확인

In [ ]:
df2 =pd.read_csv('20240611.csv')
print(df2)

         ID  Reached.on.Time_Y.N
0      8285                 0.33
1     10192                 0.39
2      8675                 0.32
3      5753                 0.38
4       448                 0.93
...     ...                  ...
2195   7193                 0.48
2196   4445                 0.29
2197   7390                 0.47
2198   9977                 0.34
2199   5696                 0.60

[2200 rows x 2 columns]


In [ ]:
you = pd.read_csv("20240611.csv")
print("당신의 score는?:",roc_auc_score(y_test['Reached.on.Time_Y.N'], you['Reached.on.Time_Y.N']))

당신의 score는?: 0.7334913864913323
